In [30]:
# -*- coding: utf-8 -*-
"""
Created on Thu May 24 13:32:33 2018

@author: cheng-man wu

LinkedIn: www.linkedin.com/in/chengmanwu
Github: https://github.com/wurmen
"""
'''==========Solving job shop scheduling problem by NSGA-II algorithm in python======='''
# importing required modules
#import os
import pandas as pd
import numpy as np
import time
''' ================= initialization setting ======================'''
num_job=10 # number of jobs
num_mc=10 # number of machines

pt_tmp=pd.read_excel("JSP_dataset.xlsx",sheet_name="Processing Time",index_col =[0])
ms_tmp=pd.read_excel("JSP_dataset.xlsx",sheet_name="Machines Sequence",index_col =[0])
job_priority_duedate_tmp=pd.read_excel("JSP_dataset.xlsx",sheet_name="Priority and Due date",index_col =[0])

# raw_input is used in python 2
population_size=int(input('Please input the size of population: ') or 20) # default value is 20
crossover_rate=float(input('Please input the size of Crossover Rate: ') or 0.8) # default value is 0.8
mutation_rate=float(input('Please input the size of Mutation Rate: ') or 0.3) # default value is 0.3
mutation_selection_rate=float(input('Please input the mutation selection rate: ') or 0.4)
num_mutation_jobs=round(num_job*num_mc*mutation_selection_rate)
num_iteration=int(input('Please input number of iteration: ') or 1000) # default value is 1000

# speed up the data search
# Below code can also be  written "pt = pt_tmp.as_matrix().tolist()"
pt=[list(map(int, pt_tmp.iloc[i])) for i in range(num_job)]
ms=[list(map(int,ms_tmp.iloc[i])) for i in range(num_job)]
job_priority_duedate=[list(job_priority_duedate_tmp.iloc[i]) for i in range(num_job)]
start_time = time.time()
'''===========function==============='''
'''-------non-dominated sorting function-------'''      
def non_dominated_sorting(population_size,chroms_obj_record):
    s,n={},{}
    front,rank={},{}
    front[0]=[]     
    for p in range(population_size*2):
        s[p]=[]
        n[p]=0
        for q in range(population_size*2):
            
            if ((chroms_obj_record[p][0]<chroms_obj_record[q][0] and chroms_obj_record[p][1]<chroms_obj_record[q][1]) or (chroms_obj_record[p][0]<=chroms_obj_record[q][0] and chroms_obj_record[p][1]<chroms_obj_record[q][1])
            or (chroms_obj_record[p][0]<chroms_obj_record[q][0] and chroms_obj_record[p][1]<=chroms_obj_record[q][1])):
                if q not in s[p]:
                    s[p].append(q)
            elif ((chroms_obj_record[p][0]>chroms_obj_record[q][0] and chroms_obj_record[p][1]>chroms_obj_record[q][1]) or (chroms_obj_record[p][0]>=chroms_obj_record[q][0] and chroms_obj_record[p][1]>chroms_obj_record[q][1])
            or (chroms_obj_record[p][0]>chroms_obj_record[q][0] and chroms_obj_record[p][1]>=chroms_obj_record[q][1])):
                n[p]=n[p]+1
        if n[p]==0:
            rank[p]=0
            if p not in front[0]:
                front[0].append(p)
    
    i=0
    while (front[i]!=[]):
        Q=[]
        for p in front[i]:
            for q in s[p]:
                n[q]=n[q]-1
                if n[q]==0:
                    rank[q]=i+1
                    if q not in Q:
                        Q.append(q)
        i=i+1
        front[i]=Q
                
    del front[len(front)-1]
    return front

'''--------calculate crowding distance function---------'''
def calculate_crowding_distance(population_size,chroms_obj_record):
    l=population_size*2
    distance={m:0 for m in range(l)}
    for o in range(2):
        obj={m:chroms_obj_record[m][o] for m in range(l)}
        sorted_keys=sorted(obj, key=obj.get)
        distance[sorted_keys[0]]=distance[sorted_keys[l-1]]=999999999999
        for i in range(1,l-1):
            if len(set(obj.values()))==1:
                distance[sorted_keys[i]]=distance[sorted_keys[i]]
            else:
                distance[sorted_keys[i]]=distance[sorted_keys[i]]+(obj[sorted_keys[i+1]]-obj[sorted_keys[i-1]])/(obj[sorted_keys[l-1]]-obj[sorted_keys[0]])
    return distance            
'''----------selection----------'''
def selection(population_size,front,distance):   
    N=0
    new_pop=[]
    while N < population_size:
        for i in range(len(front)):
            N=N+len(front[i])
            if N > population_size:
                cdf={i:distance[i] for i in front[i]}
                sorted_cdf=sorted(cdf, key=cdf.get)
                sorted_cdf.reverse()
                for j in sorted_cdf:
                    if len(new_pop)==population_size:
                        break                
                    new_pop.append(j)              
                break
            else:
                new_pop.extend(front[i])
    
    population_list=[]
    for n in new_pop:
        population_list.append(total_chromosome[n])
    
    return population_list,new_pop


'''==================== main code ==============================='''
'''----- generate initial population -----'''
best_list,best_obj=[],[]
population_list=[]
for i in range(population_size):
    nxm_random_num=list(np.random.permutation(num_job*num_mc)) # generate a random permutation of 0 to num_job*num_mc-1
    population_list.append(nxm_random_num) # add to the population_list
    for j in range(num_job*num_mc):
        population_list[i][j]=population_list[i][j]%num_job # convert to job number format, every job appears m times
        
for n in range(num_iteration):           
    '''-------- two point crossover --------'''
    parent_list=population_list[:]
    offspring_list=[]
    S=list(np.random.permutation(population_size)) # generate a random sequence to select the parent chromosome to crossover
    
    for m in range(int(population_size/2)):
        
        parent_1= population_list[S[2*m]][:]
        parent_2= population_list[S[2*m+1]][:]
        child_1=parent_1[:]
        child_2=parent_2[:]
        
        cutpoint=list(np.random.choice(num_job*num_mc, 2, replace=False))
        cutpoint.sort()
    
        child_1[cutpoint[0]:cutpoint[1]]=parent_2[cutpoint[0]:cutpoint[1]]
        child_2[cutpoint[0]:cutpoint[1]]=parent_1[cutpoint[0]:cutpoint[1]]
        
        offspring_list.extend((child_1,child_2)) # append child chromosome to offspring list
        
    '''----------repairment-------------'''
    for m in range(population_size):
        job_count={}
        larger,less=[],[] # 'larger' record jobs appear in the chromosome more than m times, and 'less' records less than m times.
        for i in range(num_job):
            if i in offspring_list[m]:
                count=offspring_list[m].count(i)
                pos=offspring_list[m].index(i)
                job_count[i]=[count,pos] # store the above two values to the job_count dictionary
            else:
                count=0
                job_count[i]=[count,0]
            if count>num_mc:
                larger.append(i)
            elif count<num_mc:
                less.append(i)
                
        for k in range(len(larger)):
            chg_job=larger[k]
            while job_count[chg_job][0]>num_mc:
                for d in range(len(less)):
                    if job_count[less[d]][0]<num_mc:                    
                        offspring_list[m][job_count[chg_job][1]]=less[d]
                        job_count[chg_job][1]=offspring_list[m].index(chg_job)
                        job_count[chg_job][0]=job_count[chg_job][0]-1
                        job_count[less[d]][0]=job_count[less[d]][0]+1                    
                    if job_count[chg_job][0]==num_mc:
                        break        
    '''--------mutatuon--------'''   
    for m in range(len(offspring_list)):
        mutation_prob=np.random.rand()
        if mutation_rate <= mutation_prob:
            m_chg=list(np.random.choice(num_job*num_mc, num_mutation_jobs, replace=False)) # chooses the position to mutation
            t_value_last=offspring_list[m][m_chg[0]] # save the value which is on the first mutation position
            for i in range(num_mutation_jobs-1):
                offspring_list[m][m_chg[i]]=offspring_list[m][m_chg[i+1]] # displacement
            
            offspring_list[m][m_chg[num_mutation_jobs-1]]=t_value_last # move the value of the first mutation position to the last mutation position
    

                        
    
    '''--------fitness value(calculate  makespan and TWET)-------------'''
    total_chromosome=parent_list[:]+offspring_list[:] # combine parent and offspring chromosomes
    chroms_obj_record={} # record each chromosome objective values as chromosome_obj_record={chromosome:[TWET,makespan]}
    for m in range(population_size*2):
        j_keys=[j for j in range(num_job)]
        key_count={key:0 for key in j_keys}
        j_count={key:0 for key in j_keys}
        m_keys=[j+1 for j in range(num_mc)]
        m_count={key:0 for key in m_keys}
        d_record={} # record jobs earliness and tardiness time as d_record={job:[earliness time,tardiness time]}
        
        for i in total_chromosome[m]:
            gen_t=int(pt[i][key_count[i]])
            gen_m=int(ms[i][key_count[i]])
            j_count[i]=j_count[i]+gen_t
            m_count[gen_m]=m_count[gen_m]+gen_t
            
            if m_count[gen_m]<j_count[i]:
                m_count[gen_m]=j_count[i]
            elif m_count[gen_m]>j_count[i]:
                j_count[i]=m_count[gen_m]
            
            key_count[i]=key_count[i]+1
    
        for j in j_keys:
            if j_count[j]>job_priority_duedate[j][1]:
                job_tardiness=j_count[j]-job_priority_duedate[j][1]
                job_earliness=0
                d_record[j]=[job_earliness,job_tardiness]
            elif j_count[j]<job_priority_duedate[j][1]:
                job_tardiness=0
                job_earliness=job_priority_duedate[j][1]-j_count[j]
                d_record[j]=[job_earliness,job_tardiness]
            else:
                job_tardiness=0
                job_earliness=0
                d_record[j]=[job_earliness,job_tardiness]
        
        twet=sum((1/job_priority_duedate[j][0])*d_record[j][0]+job_priority_duedate[j][0]*d_record[j][1] for j in j_keys)
        makespan=max(j_count.values())
        chroms_obj_record[m]=[twet,makespan]
                       
    
    '''-------non-dominated sorting-------'''      
    front=non_dominated_sorting(population_size,chroms_obj_record)
    print(front)
    
    '''--------calculate crowding distance---------'''
    distance=calculate_crowding_distance(population_size,chroms_obj_record)    
    print(distance)
    
    '''----------selection----------'''
    population_list,new_pop=selection(population_size,front,distance)
    new_pop_obj=[chroms_obj_record[k] for k in new_pop]    
    

    '''----------comparison----------'''
    if n==0:
        best_list=population_list[:]
        best_obj=new_pop_obj[:]
    else:            
        total_list=population_list[:]+best_list[:]
        total_obj=new_pop_obj[:]+best_obj[:]
        
        now_best_front=non_dominated_sorting(population_size,total_obj)
        now_best_distance=calculate_crowding_distance(population_size,total_obj)
        best_list,best_pop=selection(population_size,now_best_front,now_best_distance)
        best_obj=[total_obj[k] for k in best_pop]
'''----------result----------'''
print(best_list)
print(best_obj)
print('the elapsed time:%s'% (time.time() - start_time))




Please input the size of population: 
Please input the size of Crossover Rate: 
Please input the size of Mutation Rate: 
Please input the mutation selection rate: 
Please input number of iteration: 
{0: [7], 1: [17, 29, 35, 36], 2: [5, 25], 3: [11, 38], 4: [23, 12], 5: [19, 32, 39], 6: [2, 10, 34], 7: [22, 27], 8: [13, 14], 9: [0, 9, 24], 10: [4, 21], 11: [3], 12: [30, 33], 13: [16, 20, 28, 37], 14: [31], 15: [6, 8], 16: [1], 17: [18], 18: [26], 19: [15]}
{0: 0.029280456032132387, 1: 0.10252394241659304, 2: 0.024988481102657764, 3: 0.029481729339827335, 4: 0.0648534593579522, 5: 0.148697412011163, 6: 0.23748385665384208, 7: 999999999999, 8: 0.15471620814924492, 9: 0.07539993206215151, 10: 0.03862658531780401, 11: 0.13479047497311808, 12: 0.06905458025138085, 13: 0.05131599179665343, 14: 0.021701683664190296, 15: 999999999999, 16: 0.14767634452217993, 17: 0.14781367950468605, 18: 0.15513713095277024, 19: 0.026571751853360182, 20: 0.047456019431197524, 21: 0.05947161432921103, 22: 0.1163

{0: [0, 37], 1: [1, 2], 2: [3, 4, 5], 3: [6, 7, 8, 9, 10, 11, 12], 4: [13, 14, 31, 15], 5: [16, 17, 19, 18, 30], 6: [29, 32], 7: [20, 24], 8: [25, 35], 9: [28], 10: [21, 22, 34], 11: [26], 12: [27, 39], 13: [38, 33], 14: [23], 15: [36]}
{0: 999999999999, 1: 0.2077779374621187, 2: 0.12236367253403373, 3: 0.15439799218426425, 4: 0.04096136554194449, 5: 0.0744589768143483, 6: 0.06526829957200747, 7: 0.05208063575865264, 8: 0.005891314236147459, 9: 0.1389750227771351, 10: 0.045992742704729424, 11: 0.03963778257227876, 12: 0.06886664316751409, 13: 0.045318617416582385, 14: 0.04259078084891633, 15: 0.11887555470993, 16: 0.03349252008861235, 17: 0.2031302319098596, 18: 0.03573947956631707, 19: 0.0562894746988169, 20: 0.1394337745677291, 21: 0.12901839499885726, 22: 0.24288588031386313, 23: 0.051697266334519584, 24: 0.08944032708550045, 25: 0.07691974630348689, 26: 0.1707425841045004, 27: 0.1603712514870116, 28: 0.10037848911388958, 29: 0.08431908231720914, 30: 0.07416978443645347, 31: 0.03003

{0: [0, 1, 2], 1: [3, 4, 5], 2: [6, 7, 8, 9, 10], 3: [11, 12, 32, 13, 14, 15, 16, 26], 4: [18, 19, 17], 5: [23, 24], 6: [37, 22], 7: [25, 29, 39, 33, 35], 8: [38], 9: [20, 34], 10: [30, 31], 11: [28, 36], 12: [21, 27]}
{0: 999999999999, 1: 999999999999.0387, 2: 0.04559073189784335, 3: 0.051915884084983446, 4: 0.15311428932966364, 5: 0.07075016985973975, 6: 0.06492174055888658, 7: 0.11446015422499331, 8: 0.17420482574715135, 9: 0.039358093487435165, 10: 0.01904989442260705, 11: 0.08966852807458334, 12: 0.06727801899529848, 13: 0.029523097617621605, 14: 0.052928244451945126, 15: 0.03443549870346387, 16: 0.040087519069204464, 17: 0.10253268873522989, 18: 0.09611067936954643, 19: 0.10198166991703585, 20: 0.06924336399456815, 21: 0.20668935972516098, 22: 0.047663093681149495, 23: 0.26176160140692945, 24: 0.0651037202307024, 25: 0.09101069378254917, 26: 0.019211732165143255, 27: 0.20743566798021956, 28: 0.1640511114615228, 29: 0.14216678256571877, 30: 0.16036212461433896, 31: 0.0901892453580

{0: [0], 1: [1, 2, 3, 4, 5, 6, 7, 8, 9], 2: [10, 11, 12], 3: [13, 14, 15, 16], 4: [17, 19, 18], 5: [35, 26, 33], 6: [20, 36], 7: [23, 34], 8: [21], 9: [32], 10: [31, 39], 11: [27, 29], 12: [22, 25], 13: [37, 28], 14: [30, 38], 15: [24]}
{0: 999999999999, 1: 0.050952066248541095, 2: 0.017274606316516103, 3: 0.04005218855762466, 4: 0.05091685170893663, 5: 0.018506128549752962, 6: 0.04598346154439527, 7: 0.024597095796105115, 8: 0.032229824831434835, 9: 0.05440965685646641, 10: 0.022506109175477487, 11: 0.03615447899164789, 12: 0.049947169221021374, 13: 0.025188513494362234, 14: 0.015460833287588547, 15: 0.027266709505342435, 16: 0.040545981434421126, 17: 0.030428591264139184, 18: 0.04104458200180041, 19: 0.04261884951964354, 20: 0.08635634817869253, 21: 0.0673902958559501, 22: 0.07653653252851159, 23: 0.0913788779495989, 24: 999999999999, 25: 0.09606512052772656, 26: 0.0911554638506516, 27: 0.22395862372322628, 28: 0.1342227202128587, 29: 0.09502547537834552, 30: 0.41032828274674127, 31:

{0: [0], 1: [1, 2, 3, 4], 2: [5, 6, 9, 16, 19, 7, 8, 10, 11, 12, 13, 14, 15, 17, 18], 3: [20, 36], 4: [21, 24, 34], 5: [38, 22, 31], 6: [29, 32, 39], 7: [26, 27, 28, 35], 8: [23], 9: [25], 10: [37], 11: [33], 12: [30]}
{0: 999999999999, 1: 0.12506363632434864, 2: 0.07198006945122072, 3: 0.018294405485131584, 4: 0.01842588037167113, 5: 0.05800439875927653, 6: 0.1141283614775227, 7: 0.11585420179430436, 8: 0.07745218490210438, 9: 0.036798695407016586, 10: 0.021381401247865615, 11: 0.04818765840844474, 12: 0.012965808682289666, 13: 0.06324259342785926, 14: 0.0, 15: 0.022311608389190683, 16: 0.0, 17: 0.0, 18: 0.013678750460312781, 19: 0.04893913112506021, 20: 0.32083773175142677, 21: 0.2447031900842199, 22: 0.0486570239239826, 23: 0.0981060474239884, 24: 0.23073196614463384, 25: 0.06853912471081894, 26: 999999999999, 27: 0.15213789845346276, 28: 0.21907630604301134, 29: 0.23062046183901758, 30: 999999999999.0586, 31: 0.10602433726899349, 32: 0.15082531041402109, 33: 0.10306838422760822, 34

...

{0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 34, 35, 38], 1: [29, 30, 37], 2: [23, 27], 3: [32], 4: [24, 36], 5: [21, 22, 25, 31, 39], 6: [28], 7: [33], 8: [26]}
{0: 999999999999.1351, 1: 999999999999, 2: 0.3440995067237945, 3: 0.0, 4: 0.11476335670839079, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.13515494908076742, 22: 0.0805168662838197, 23: 0.06479492360399516, 24: 0.16355876854703802, 25: 0.11620496193169638, 26: 999999999999, 27: 0.11315974926186073, 28: 0.13963791032275, 29: 0.45601631175868695, 30: 0.2640779296069208, 31: 0.2062293654294548, 32: 0.10259501383956757, 33: 0.32294491046027185, 34: 0.0, 35: 0.13953958630531083, 36: 0.20986476009937072, 37: 0.378612083203749, 38: 0.2282768702219043, 39: 0.01834059051688367}
{0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 29], 1: [27], 2: [33], 3: [36, 37], 4: [

{0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 27, 32, 36, 38, 39], 1: [24], 2: [29, 37], 3: [26, 28], 4: [31, 33], 5: [34], 6: [22], 7: [21], 8: [23, 30], 9: [25], 10: [35]}
{0: 999999999999.0397, 1: 999999999999, 2: 0.12050144134325164, 3: 0.10715779293760944, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.3082159939958105, 21: 0.25317832462471823, 22: 0.2675994476693106, 23: 0.23217087542833195, 24: 0.47633988100764807, 25: 0.10555421039663486, 26: 0.17105011932465772, 27: 0.0, 28: 0.18292842721847366, 29: 0.5456179206227189, 30: 0.12608321031222264, 31: 0.05429469169824628, 32: 0.0, 33: 0.15473320881942432, 34: 0.20032718295740826, 35: 999999999999, 36: 0.0, 37: 0.4716058980578577, 38: 0.0, 39: 0.0964341696175407}
{0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 24, 28, 29, 36], 1: [25, 30], 2: [32], 3: [20, 35, 38], 4: [

{0: 999999999999.1163, 1: 999999999999, 2: 0.11711856010423427, 3: 0.16982516829382302, 4: 0.09851390894144359, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.33732754400651155, 21: 0.12305605671505035, 22: 0.0, 23: 0.0, 24: 0.11052200404728346, 25: 999999999999, 26: 0.15059903931412955, 27: 0.07789469316273692, 28: 0.06038572264542068, 29: 0.34285801857175124, 30: 0.27645596479196155, 31: 0.25192910160640414, 32: 0.12004753917795592, 33: 0.07071044166396823, 34: 0.37520596302369164, 35: 0.20868116436900486, 36: 0.35704438640327774, 37: 0.19155266153583234, 38: 0.10828453975033643, 39: 0.17562156707243015}
{0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 28, 29, 34, 37], 1: [32, 36], 2: [24, 23, 27], 3: [30], 4: [20, 22], 5: [21, 25], 6: [33, 35, 38], 7: [31, 39], 8: [26]}
{0: 999999999999.1425, 1: 999999999999, 2: 0.3052717754873179, 3: 0.12451606069467586, 4: 0.0, 5: 0.0

{0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 26, 29, 33, 38, 39], 1: [23], 2: [37], 3: [36], 4: [30], 5: [24, 32, 34], 6: [28, 35], 7: [27], 8: [21, 25, 31]}
{0: 999999999999.1302, 1: 999999999999, 2: 0.0, 3: 0.0, 4: 0.1095447314945363, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 999999999999.0677, 22: 0.0, 23: 0.4812711835205784, 24: 0.06415573041769136, 25: 999999999999, 26: 0.39188596399414555, 27: 0.23908543213086098, 28: 0.3718254790245184, 29: 0.1251697314945363, 30: 0.19630050219090484, 31: 0.2709305120382805, 32: 0.16139368044103058, 33: 0.0, 34: 0.07236777333013283, 35: 0.4752229606867884, 36: 0.2510266169885239, 37: 0.16812834287767192, 38: 0.0, 39: 0.1343583462522821}
{0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 27, 30, 35, 36, 37, 38], 1: [28], 2: [24], 3: [20, 29], 4: [26, 33], 5: [25, 39], 6: [31

{0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 26, 33, 34, 35, 38], 1: [36, 39], 2: [31, 37], 3: [25, 27, 30], 4: [29, 32], 5: [21], 6: [24], 7: [28]}
{0: 999999999999.1287, 1: 999999999999, 2: 0.19226225233383798, 3: 0.09943933680340139, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0, 8: 0.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 0.0, 16: 0.0, 17: 0.0, 18: 0.0, 19: 0.0, 20: 0.0, 21: 0.224710513511238, 22: 0.0, 23: 0.0, 24: 0.5553506683555667, 25: 0.19694553540215656, 26: 0.0, 27: 0.20505927550596295, 28: 999999999999, 29: 0.0980284910292292, 30: 0.36870405865847417, 31: 0.08530469914544006, 32: 0.07594952367194784, 33: 0.0, 34: 0.0, 35: 0.15606739343145803, 36: 0.20318014256062983, 37: 0.2921769934324061, 38: 0.13200037501366987, 39: 0.33478627134594846}
{0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 24, 25, 29, 31, 33], 1: [22], 2: [26], 3: [27], 4: [28], 5: [23, 39], 6: [32, 34], 7: [36, 38], 8: [20], 9: [30,

取其中一個解來畫排程甘特圖

In [31]:
'''--------plot gantt chart-------'''
import pandas as pd
import plotly.plotly as py
import plotly.figure_factory as ff
import datetime

m_keys=[j+1 for j in range(num_mc)]
j_keys=[j for j in range(num_job)]
key_count={key:0 for key in j_keys}
j_count={key:0 for key in j_keys}
m_count={key:0 for key in m_keys}
j_record={}
for i in best_list[0]:
    gen_t=int(pt[i][key_count[i]])
    gen_m=int(ms[i][key_count[i]])
    j_count[i]=j_count[i]+gen_t
    m_count[gen_m]=m_count[gen_m]+gen_t
    
    if m_count[gen_m]<j_count[i]:
        m_count[gen_m]=j_count[i]
    elif m_count[gen_m]>j_count[i]:
        j_count[i]=m_count[gen_m]
    
    start=j_count[i]-pt[i][key_count[i]]
    print(j_count[i],start)
    start_time=str(datetime.timedelta(seconds=start)) # convert seconds to hours, minutes and seconds
    end_time=str(datetime.timedelta(seconds=j_count[i]))
    print(start_time,end_time)    
    j_record[(i,gen_m)]=[start_time,end_time]
    
    key_count[i]=key_count[i]+1
        

df=[]
for m in m_keys:
    for j in j_keys:
        df.append(dict(Task='Machine %s'%(m), Start='2018-07-14 %s'%(str(j_record[(j,m)][0])), Finish='2018-07-14 %s'%(str(j_record[(j,m)][1])),Resource='Job %s'%(j+1)))
    
fig = ff.create_gantt(df, index_col='Resource', show_colorbar=True, group_tasks=True, showgrid_x=True, title='Job shop Schedule')
py.iplot(fig, filename='GA_job_shop_scheduling1', world_readable=True)

14 0
0:00:00 0:00:14
20 14
0:00:14 0:00:20
81 0
0:00:00 0:01:21
103 81
0:01:21 0:01:43
63 20
0:00:20 0:01:03
98 14
0:00:14 0:01:38
164 103
0:01:43 0:02:44
190 164
0:02:44 0:03:10
188 98
0:01:38 0:03:08
188 103
0:01:43 0:03:08
92 63
0:01:03 0:01:32
259 190
0:03:10 0:04:19
190 188
0:03:08 0:03:10
283 188
0:03:08 0:04:43
280 259
0:04:19 0:04:40
268 190
0:03:10 0:04:28
334 259
0:04:19 0:05:34
168 92
0:01:32 0:02:48
292 283
0:04:43 0:04:52
345 334
0:05:34 0:05:45
201 188
0:03:08 0:03:21
242 190
0:03:10 0:04:02
353 292
0:04:52 0:05:53
337 242
0:04:02 0:05:37
360 353
0:05:53 0:06:00
424 360
0:06:00 0:07:04
373 337
0:05:37 0:06:13
354 283
0:04:43 0:05:54
453 354
0:05:54 0:07:33
502 453
0:07:33 0:08:22
442 373
0:06:13 0:07:22
462 453
0:07:33 0:07:42
513 502
0:08:22 0:08:33
500 424
0:07:04 0:08:20
472 424
0:07:04 0:07:52
560 513
0:08:33 0:09:20
337 268
0:04:28 0:05:37
524 472
0:07:52 0:08:44
518 442
0:07:22 0:08:38
609 524
0:08:44 0:10:09
658 609
0:10:09 0:10:58
384 353
0:05:53 0:06:24
572 500
0

In [26]:
print(j_record[(2,5)])


['0:26:58', '0:27:31']


In [17]:
print(pt)

[[29, 78, 9, 36, 49, 11, 62, 56, 44, 21], [43, 90, 75, 11, 69, 28, 46, 46, 72, 30], [91, 85, 39, 74, 90, 10, 12, 89, 45, 33], [81, 95, 71, 99, 9, 52, 85, 98, 22, 43], [14, 6, 22, 61, 26, 69, 21, 49, 72, 53], [84, 2, 52, 95, 48, 72, 47, 65, 6, 25], [46, 37, 61, 13, 32, 21, 32, 89, 30, 55], [31, 86, 46, 74, 32, 88, 19, 48, 36, 79], [76, 69, 76, 51, 85, 11, 40, 89, 26, 74], [85, 13, 61, 7, 64, 76, 47, 52, 90, 45]]
